<a href="https://colab.research.google.com/github/A-Wadhwani/ME498-Project/blob/main/05_Extreme_Learning_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Open drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import tensorflow as tf
import pandas as pd
import random
import time
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error

import lightgbm as lgb

In [3]:
# Reading training and testing dataset files
path = "drive/My Drive/ASHRAE_DATA/"
x_train = pd.read_csv(path + "x_train.csv", index_col=0)
x_test = pd.read_csv(path + "x_test.csv", index_col=0)
y_train = pd.read_csv(path + "y_train.csv", index_col=0)
y_test = pd.read_csv(path + "y_test.csv", index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# ELM
For Extreme Learning Machine, I tested several implementations available online

##1. ELM implementation from: https://github.com/ivallesp/simplestELM

In [4]:

class ELMRegressor():
    def __init__(self, n_hidden_units):
        self.n_hidden_units = n_hidden_units

    def fit(self, X, labels):
        X = np.column_stack([X, np.ones([X.shape[0], 1])])
        self.random_weights = np.random.randn(X.shape[1], self.n_hidden_units)
        G = np.tanh(X.dot(self.random_weights))
        self.w_elm = np.linalg.pinv(G).dot(labels)

    def predict(self, X):
        X = np.column_stack([X, np.ones([X.shape[0], 1])])
        G = np.tanh(X.dot(self.random_weights))
        return G.dot(self.w_elm)

In [5]:
# Scaling data
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler() 
y_scaler = StandardScaler()

x_train_scaled = x_scaler.fit_transform(x_train)
y_train_scaled = y_scaler.fit_transform(y_train)

x_test_scaled = x_scaler.fit_transform(x_test)
y_test_scaled = y_scaler.transform(y_test)

In [6]:
elm = ELMRegressor(n_hidden_units=300)
start_time = time.time()
elm.fit(x_train_scaled, y_train_scaled)
end_time = time.time()
print("Training Time: ", end_time-start_time, " seconds")

Training Time:  114.05731558799744  seconds


In [7]:
# Make predictions
start_time = time.time()
y_predict = y_scaler.inverse_transform(elm.predict(x_test_scaled))
end_time = time.time()
print("Prediction Time: ", end_time-start_time, " seconds")

Prediction Time:  6.041177988052368  seconds


In [8]:
# Evaluate accuracy
print("MSE: ", mean_squared_error(y_test, y_predict))
print("R^2 Score: ", r2_score(y_test, y_predict))

MSE:  55038.95389868501
R^2 Score:  0.34516163859352567


##2. ELM from Sklearn Extensions

https://github.com/wdm0006/sklearn-extensions

In [9]:
!pip install sklearn-extensions

  Created wheel for sklearn-extensions: filename=sklearn_extensions-0.0.2-py2.py3-none-any.whl size=24578 sha256=a5ef22d66dd1accb5c6703d28a1cc3bcbd415fd830fab02bc5bf7bdc2b12800d
  Stored in directory: /root/.cache/pip/wheels/b5/26/b2/09100728168b45e4de834b943a588028744fdfaa9ac979ffb2
Successfully built sklearn-extensions


In [14]:
from sklearn_extensions import extreme_learning_machines as skelm

In [22]:
# Here, we will evaluate the ELM Regressor with different n_hidden, alpha values and activation functions

hidden_units = [10, 20, 40, 50, 75, 100]
alphas = [0.1, 0.25, 0.5]
functions = ['tanh', 'sine', 'tribas','sigmoid', 'hardlim', 'gaussian','multiquadric']

for alpha in alphas:
  for n_hidden in hidden_units:
    for function in functions:
      # Make model
      model = skelm.ELMRegressor(n_hidden=n_hidden, alpha=alpha, activation_func=function)
      name = "(" + function + ", " + str(alpha) + ", " + str(n_hidden) + ")"
      
      # Fit Model
      start_time = time.time()
      model.fit(x_train_scaled, y_train_scaled)
      end_time = time.time()
      print("Training Time for ", name," : ", end_time-start_time, " seconds")

      # Evaluate Model
      start_time = time.time()
      y_predict = y_scaler.inverse_transform(model.predict(x_test_scaled))
      end_time = time.time()
      print("Prediction Time for ", name," : ", end_time-start_time, " seconds")
      print("MSE for ", name," : ", mean_squared_error(y_test, y_predict))
      print("R^2 Score for ", name," : ", r2_score(y_test, y_predict))
      print("")

Training Time for  (tanh, 0.1, 10)  :  1.74949312210083  seconds
Prediction Time for  (tanh, 0.1, 10)  :  0.45459675788879395  seconds
MSE for  (tanh, 0.1, 10)  :  71632.07801088756
R^2 Score for  (tanh, 0.1, 10)  :  0.14774120389103063

Training Time for  (sine, 0.1, 10)  :  1.6443064212799072  seconds
Prediction Time for  (sine, 0.1, 10)  :  0.45488643646240234  seconds
MSE for  (sine, 0.1, 10)  :  79455.1528987384
R^2 Score for  (sine, 0.1, 10)  :  0.05466440686195806

Training Time for  (tribas, 0.1, 10)  :  1.3626832962036133  seconds
Prediction Time for  (tribas, 0.1, 10)  :  0.34754514694213867  seconds
MSE for  (tribas, 0.1, 10)  :  80529.87146193594
R^2 Score for  (tribas, 0.1, 10)  :  0.0418777004831834

Training Time for  (sigmoid, 0.1, 10)  :  1.61165452003479  seconds
Prediction Time for  (sigmoid, 0.1, 10)  :  0.4308745861053467  seconds
MSE for  (sigmoid, 0.1, 10)  :  73308.51543572628
R^2 Score for  (sigmoid, 0.1, 10)  :  0.12779541171077424

Training Time for  (hardlim

As visible, the Extreme Learning Machine models fail to make accurate predictions, with the best model having an MSE of 52733.81 and $R^2$ of 0.372.

In comparison, a catboost model on the same data has an $R^2$ of 0.99

In [24]:
model = skelm.ELMRegressor(n_hidden=100, alpha=0.5, activation_func='sigmoid')

# Fit Model
start_time = time.time()
model.fit(x_train_scaled, y_train_scaled)
end_time = time.time()
print("Training Time: ", end_time-start_time, " seconds")

# Evaluate Model
start_time = time.time()
y_predict = y_scaler.inverse_transform(model.predict(x_test_scaled))
end_time = time.time()
print("Prediction Time: ", end_time-start_time, " seconds")
print("MSE: ", mean_squared_error(y_test, y_predict))
print("R^2 Score: ", r2_score(y_test, y_predict))

Training Time:  38.273001194000244  seconds
Prediction Time:  2.8420283794403076  seconds
MSE:  52750.91368285495
R^2 Score:  0.372384112852835
